# Bus I2C

## Objectif 

L'objectif de ce notebook a deux objectifs le premier est d'expliquer comment fonctionne le bus I2C. Pour ensuite, voir comment mettre en place l'architecture électronique de se système. 

## Import and Formatting

The goal of this section is to import all the necessary files and libraries required for the subsequent data analysis.  
It also includes setting up the formatting parameters for the plots and visualizations.

### Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np

### Formatting

Adjusting Plotly chart settings for clarity and consistency.

In [2]:
# ----  Formatting charts
%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2.0
mpl.rcParams['axes.edgecolor']  = "#bcbcbc"
mpl.rcParams['patch.linewidth'] = 0.5
mpl.rcParams['legend.fancybox'] = True
mpl.rcParams['axes.facecolor']  = "#eeeeee"
mpl.rcParams['axes.labelsize']  = "large"
mpl.rcParams['axes.grid']       = True
mpl.rcParams['grid.linestyle']  = "--"
mpl.rcParams['patch.edgecolor'] = "#eeeeee"
mpl.rcParams['axes.titlesize']  = "x-large"

## History

The $I^2C$ (*Inter-Integrated Circuit*) standard was developed by Philips in 1982. It is now maintained by NXP Semiconductors, the former semiconductor division of Philips.

Several major revisions have been released over the years:
* version 1.0 in 1992,
* version 2.0 in 1998,
* version 2.1 in 2000,
* version 3.0 in 2007,
* version 4.0 in February 2012,
* version 5.0 in October 2012,
* version 6.0 in April 2014,

and the current version 7.0, released in October 2021.

It is important to note that $I^2C$ is a proprietary bus owned by NXP. Using the bus as a communication standard in your products is free, but if you plan to design or implement your own $I^2C$ controller (for example, a hardware IP core), you may need to obtain a license from NXP.

This notebook is based on the $I^2C$ specification version 7.0.

## Quick Overview

### Topology

As its name suggests, the $I^2C$ architecture is a **bus-based architecture**. The [](#bus_topologie) illustrates this, with the white dots representing the devices connected on the bus. All elements are therefore **connected to the same physical cable(s)**.

```{figure} ./images/bus_topologie.svg
:name: bus_topologie
:align: center
:width: 400px

Visualization of a bus topology
```

The [](#I2C_modelisation_electrique_simple) provides a simplified view of the system. Two wires are essential, even though they will not be studied further in detail:

* **VDD**: the reference voltage used to bias the lines.
* **GND**: the ground reference of the circuit.

```{important}
All devices on the bus must share the same ground reference; otherwise, communication issues may occur.
```

```{note}
The system includes two pull-up resistors on the *SDA* and *SCL* lines. This means that when nothing is driving the bus, both lines remain at a logical **high (1)** level.
```

```{figure} ./images/I2C_modélisation_electrique_simple.svg
:name: I2C_modelisation_electrique_simple
:align: center
:width: 400px

First simplified view of an $I^2C$ architecture
```

### SCL Signal

The **SCL** line corresponds to the **clock signal** of the system. It synchronizes all data transfers.

### SDA Signal

The **SDA** line is the **data line**, where information is transmitted.

### Master / Slave

As shown in [](#I2C_modelisation_electrique_simple), an $I^2C$ bus involves two types of roles: **master** and **slave**.

A **master** can:

* select the device it wants to communicate with,
* decide whether it wants to read or write,
* initiate and control the communication process.

A **slave** simply responds to the master and follows its instructions. It never initiates communication on its own.

## Communication Visualization

### Fundamental Timing Diagram

A data bit is considered valid on the bus only when the **SCL** line is **high**.
The transmitter must therefore place the data on the **SDA** line while **SCL is low**, and hold it stable while **SCL is high**, as shown in [](#chronogramme-fondamental-I2C).
This behavior is common to most synchronous communication buses.

```{figure} ./images/chronogramme-fondamental.svg
:name: chronogramme-fondamental-I2C
:align: center
:width: 400px

Fundamental timing diagram for the $I^2C$ bus
```

We can now examine how data transmission starts.

### Start

There may be several potential masters on the same bus, but only one can control it at any given time.
To take control of the bus, a master must generate a **START condition**.

Before communication begins, both **SDA** and **SCL** are held high thanks to the pull-up resistors.

To initiate communication, the master pulls **SDA** low **while SCL remains high**.
This falling edge on SDA, with SCL held high, defines the **START condition**.
After issuing START, the master controls the bus and begins generating the clock pulses on **SCL**.

This is illustrated in [](#I2C_start).

```{note}
In most systems, microcontrollers are the devices that act as I²C masters.
```

```{figure} ./images/I2C_start.svg
:name: I2C_start
:align: center
:width: 500px

START condition generated by the master
```

```{important}
**Resume Start**

* SCL   : $1 → 0$  
* SDA   : $1 → 0$ 
* The master's control of the clock
```

### Transmission d'octet

Avant de voir comment le maitre peut choisir avec qu'elle esclave parler, il faut tout d'abord, expliquer comment transmettre des données au sein de la ligne. Tout d'abord, les données sont passer par octet, donc il va y avoir des pacqué avec 8 bits. 

Il va y avoir 7 bits de poids fort et un composant qui ne nomme ACK qui est un bit d'aquitement. 

Le maitre va envoyer les 7 bit sur la ligne. Puis le dernier bit le 8ème, il va le conserver au niveau 1. Et c'est l'esclave qui va forcer la ligne à 0, pour dire qu'il a bien recus toutes les informations. Puis ensuite lme maitre peut continuer de transmettre les informations jusqu'à qu'il lache la ligne.

La [](#I2C_simple_vision_maitre_esclave) résume toutes les informations ditent.

```{figure} ./images/I2C_simple_vision_maitre_esclave.svg
:name: I2C_simple_vision_maitre_esclave
:align: center
:width: 500px

Transmission d'un octet sur un ligne I2C
```


* SCL : Horloge imposée par le maître.
* SDAM : Niveaux de SDA imposés par le maître.
* SDAE : Niveaux de SDA imposés par l'esclave.
* SDAR : Niveaux de SDA réels résultants.


### Transmission d'une addresse

Après que le maitre est prit le controle de la ligne, il faut qu'il puisse choisir avec qu'elle composant il veut communiquer pour cela, il va envoyer une adresse qui correspond au composant ou le maitre veut soit écrire ou lire des informations.

```{danger}
Chaque composant de la ligne doit avoir une adresse et chaque adresse doit être unique. Sinon, cela peut poser des problèmes sur la ligne.
```

L'addresse d'un composant est codé sur 6 bits. Ce qui donne au maximum 64 appareils qui sont connecté sur la même ligne. Mais attention, toutes les addresses ne sont pas disponible. Ce qui réduit le nombre de composant possible. Une représentation de la trame est donnée sur [](#I2C_adresse.svg)

```{figure} ./images/I2C_adresse.svg
:name: I2C_adresse
:align: center
:width: 500px

Addressage d'un maitre à un composant
```

Ensuite, il est possible de voir qu'entre ACK et le bit 0, il y a `R/W` donc le maitre demande au composant soit de lire une information ou soit d'écrire une information.
* *0* : Ecriture (maître -> esclave)
* *1* : Lecture (esclave -> maître)

```{important}
Il est important de noté que pour bien différencier les données au adresses. 
* Les *données* sont notées : **D**
* Les *adressses* sont notées : **A** 
```

```{note}
**Les adresses réservées**

Il y a deux types d'addresse qui sont réservé :
* 00000XXX
* 111111XX

Ces adresses sont réversé à des modes de fonctionnement particuliers. Une explication va être donnée plus tard dans ce chapitre.
```

Dans la suite l'objectif est de comprendre comment sont envoyer les données suite au fait que l'addresse à été sélectionné.

### Lecture / Ecriture

#### Ecriture

Pour l'écrire, tout d'abord (cf. [](#I2C_simple_ecriture)), le maitre envoie l'addresse du composant ou il veut écrire. Ensuite, le 7ème bit, il le met à 0, ce qui ,veut dire qu'il veut écrire. Ensuite, l'esclave valide qu'il est prêt. Enfin, le maitre envoie les données sur le bus pour le composant. Ensuite, 

```{note}
L'écriture d'un octet dans certains composants (Mémoire, microcontroleur, ....) peut prendre un certain temps. Il est donc possible quer le maitre soit obligé d'attendre l'acquittement ACK avant de passer à la suite. 
```

```{figure} ./images/I2C_simple_ecriture.svg
:name: I2C_simple_ecriture
:align: center
:width: 500px

I2C écriture des données (adresse + exemple)
```


#### Lecture

La lecture des données par le maitre (cf. [](#I2C_simple_lecture)), se fait de la même manière que l'écriture. Tout d'abord, le maitre envoie l'addresse, du composant qu'il veut lire des informations. Ensuite, le composant va envoyer les données pour la lecture. La seule différence, vient après c'est l'esclave qui les bits sur la ligne et non, le maitre. Enfin, pour savoir si l'esclave doit continuer a écrire ou pas sur la ligne, c'est le maitre qui choisi, grâce au *ACK*.
* Si le maître positionne ACK à '0', alors il veut avoir le bit suivant
* Si le maitre laisse ACK à '1', alors il va envoyer la condition d'arrêt.

```{figure} ./images/I2C_simple_lecture.svg
:name: I2C_simple_lecture
:align: center
:width: 500px

I2C lecture des données (adresse + exemple)
```

### Fin de la trame

Tout d'abord, comme la condition de départ, le lancement ou l'arrêt d'une communication ne peut se faire que par le maitre. 

La condition d'arrêt est très simple. Le maitre va passer le signal de SDA à '1' mais aussi arrêté l'horloge du système en la laissant à 1. Comme cela est montrer sur la [](#I2C_simple_end)

```{figure} ./images/I2C_simple.svg
:name: I2C_simple_end
:align: center
:width: 500px

Fin de communication entre le maitre est l'esclave
```

## Collision

Comme vu précedement, il est possible d'avoir plusieurs maitre sur la même ligne. Un problème qui peut venir est lorsque deux maitre veut prendre en même temps la ligne. Donc cela va créer une colision. 

Il y a trois cas possibles :

* Les différents maîtres envoient les mêmes données au même moment. Les données ne sont pas corrompues, la transmission s'effectue normalement, comme si un seul maître avait parlé. Ce cas est rare.

* Un maître impose un ``0`` sur le bus. Il relira forcément ``0`` et continuera à transmettre. Il ne peut pas alors détecter un éventuel conflit.

* Un maître cherche à appliquer un ``1`` sur le bus. S'il ne relit pas un niveau ``1``, c'est qu'un autre maître a pris la parole en même temps. Le premier perd alors immédiatement le contrôle du bus, pour ne pas perturber la transmission du second. Il continue néanmoins à lire les données au cas celles-ci lui auraient été destinées.



## Referance 
**Cours**
* https://www.technologuepro.com/cours-systemes-embarques/cours-systemes-embarques-Bus-I2C.htm
* https://www.electronique-mixte.fr/i2c/
* 

**Documentation**
* [*Documentation officiel I2C*](https://www.nxp.com/docs/en/user-guide/UM10204.pdf)